In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import matplotlib
import torch

import matplotlib.pyplot as plt
from omegaconf import OmegaConf
from flipper_training.experiments.ppo.eval import log_from_eval_rollout, PPOExperimentConfig, prepare_env
from torchrl.envs import StepCounter, VecNorm, TransformedEnv, set_exploration_type

In [2]:
cfg_dict = OmegaConf.load("../cross_eval_configs/file_ridge.yaml")
cfg_dict["heightmap_gen_opts"]["file_path"] = "../heightmap_files/ridge.pt"
cfg = PPOExperimentConfig(**cfg_dict)

In [3]:
env, device, rng = prepare_env(cfg, mode="eval")
env = TransformedEnv(env, StepCounter())

2025-05-22 20:35:17,533 [RobotModelConfig][INFO]: Loading robot model from cache: /mnt/personal/korcadav/flipper_training/.robot_cache/marv_vx0.010_dp384_b512_whl0.02_trck0.05_eaecc2d5466de1eb8911703837d75c759b5c075158ced88ea318e932700dabb2 (robot_config.py:155)
Initializing start/goal position cache: 100%|██████████| 16/16 [00:00<00:00, 95.57it/s]
2025-05-22 20:35:18,135 [environment][INFO]: Environment: Compiling engine with options {'options': {'triton.cudagraphs': True}, 'fullgraph': True} (env.py:134)
2025-05-22 20:35:24,755 [environment][INFO]: Engine compiled successfully, testing correctness with atol=0.1, rtol=1e-06 (env.py:148)
2025-05-22 20:35:25,163 [environment][INFO]: Compiled engine passed correctness test (env.py:155)
2025-05-22 20:35:25,495 [environment][INFO]: Compiled engine takes 0.331204513 ms per step (env.py:161)
2025-05-22 20:35:25,496 [environment][INFO]: Resetting environment (env.py:271)


                      Environment Summary                       
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                 Key ┃                                  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    Number of robots │                                     16 │
│        Observations │            LocalStateVector, Heightmap │
│              Reward │ PotentialGoalWithPenaltiesConfigurable │
│           Objective │              RandomNavigationObjective │
│   Physics frequency │                              142.86 Hz │
│   Engine iters/step │                                      4 │
│ Effective frequency │                               35.71 Hz │
└─────────────────────┴────────────────────────────────────────┘

2025-05-22 20:35:31,664 [environment][INFO]: Resetting environment (env.py:271)
2025-05-22 20:35:31,975 [torchrl][INFO] check_env_specs succeeded!


In [4]:
policies = [
    "final_coarse_gaussian_terrain_thesis_98_2025-05-12_12-07-12",
    "final_mixed_objective_training_666_2025-05-16_19-08-25",
    "mixed_training_smoothness_2025-05-20_06-47-01",
]

from copy import deepcopy

p_root = Path("../runs/ppo/")
p_sds = [torch.load(p_root / f"{policy}/weights/policy_final.pth", map_location=device) for policy in policies]
vecnorm_sds = [torch.load(p_root / f"{policy}/weights/vecnorm_final.pth", map_location=device) for policy in policies]


In [5]:
policy_config = cfg.policy_config(**cfg.policy_opts)
actor_value_wrapper, optim_groups, policy_transforms = policy_config.create(
    env=env,
    device=device,
)
vecnorm = VecNorm(in_keys=[o.name for o in env.observations if o.supports_vecnorm], **cfg.vecnorm_opts)
vecnorm = vecnorm.to(device)

2025-05-22 20:35:32,639 [MLPPolicyConfig][INFO]: Applied orthogonal initialization to the actor and value operators. (mlp_policy.py:89)


     Policy Parameter Counts     
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃        Component ┃ Parameters ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│    Actor Encoder │     32,032 │
│       Actor Head │     13,456 │
│    Value Encoder │     32,032 │
│       Value Head │     12,737 │
│              --- │        --- │
│      Total Actor │     45,488 │
│      Total Value │     44,769 │
│              --- │        --- │
│ Total Parameters │     90,257 │
└──────────────────┴────────────┘

In [6]:
env._set_truncate_mode(False)

In [7]:
set_exploration_type("DETERMINISTIC")

In [8]:
import torch
import torch.nn.functional as F
from tqdm import trange

class IteratedWeightedPolicy:
    def __init__(self, weights, policy_callables):
        self.weights = weights
        self.policy_callables = policy_callables
        self.i = 0

    def __call__(self, tensordict):
        weight_i = self.weights[self.i]  # (batch_size, num_policy_options)
        # actions: (num_policy_options, batch_size, action_dim)
        actions = torch.stack([p(tensordict) for p in self.policy_callables], dim=0)
        # transpose weights for broadcasting: (num_policy_options, batch_size, 1)
        actions_wt = actions * weight_i.T
        # sum over policies -> (batch_size, action_dim)
        action = actions_wt.sum(dim=0)
        self.i += 1
        return action


class MultiPolicyMPPIPlanner:
    def __init__(
        self,
        policy_obj,
        vecnorm_obj: VecNorm,
        weights: list,
        vecnorm_weights: list,
        env,  # Not directly used in these specific methods
        device,
        rng,  # rng is expected to be a torch.Generator or compatible
        rollout_steps=20,
        num_samples=10,
        lambda_=1.0,
        noise_sigma=0.1,
        temperature=1.0,
    ):  # MPPI hyperparameters
        self.env = env
        self.policies_callables = []
        self.num_policy_options = len(weights)
        for i in range(self.num_policy_options):
            policy = deepcopy(policy_obj)
            wt = weights[i]
            vn_wt = vecnorm_weights[i]
            if isinstance(wt, str):  # if path
                policy.load_state_dict(torch.load(wt, map_location=device))
            else:  # if state_dict
                policy.load_state_dict(wt)
            policy = policy.to(device)
            policy.eval()
            vecnorm = deepcopy(vecnorm_obj)
            # Assuming vecnorm_obj has a load_state_dict method and can be moved to device
            # This might need adjustment based on what vecnorm_obj actually is
            if hasattr(vecnorm, "load_state_dict"):
                if isinstance(vn_wt, str):
                    vecnorm.load_state_dict(torch.load(vn_wt, map_location=device))
                else:
                    vecnorm.load_state_dict(vn_wt)
            vecnorm = vecnorm.to(device)
            vecnorm.eval()
            print(vecnorm.state_dict())
            self.policies_callables.append(lambda td: policy(vecnorm(td)))
        self.device = device
        self.rng = rng  # Expected to be a torch.Generator for torch.randn
        self.rollout_steps = rollout_steps
        self.num_samples = num_samples
        # MPPI specific parameters
        self.lambda_ = lambda_  # Cost scaling factor in importance weights
        self.noise_sigma = noise_sigma  # Standard deviation of Gaussian noise for logits
        self.temperature = temperature  # Temperature for softmax conversion of logits to policy weights
        # Mean logits for policy weights over the rollout horizon
        # Shape: (rollout_steps, num_policy_options)
        # Initialized to zeros, meaning initial policy weights are uniform
        self.mean_logits = torch.zeros(self.rollout_steps, self.env.n_robots, self.num_policy_options, device=self.device)
        # To store the perturbations from the latest call, needed for the update step
        self.last_logit_perturbations = None

    def __call__(self, tensordict):
        weights = self._get_rollout_weights()
        costs = []
        for s in trange(self.num_samples, desc="Sampling", leave=False):
            weight = weights[s]  # (rollout_steps, batch_size, num_policy_options)
            weighted_policy = IteratedWeightedPolicy(weight, self.policies_callables)
            with set_exploration_type("DETERMINISTIC"), torch.inference_mode():
                rollout = self.env.rollout(
                    auto_reset=False,
                    policy=weighted_policy,
                    tensordict=tensordict.clone(),
                    max_steps=self.rollout_steps,
                )
                rew = rollout["next", "reward"].sum(dim=1)  # (batch_size,)
                costs.append(-rew)  # shape (B, 1)
        costs = torch.stack(costs, dim=0)  # (num_samples, batch_size, 1)
        # aggregate costs across batch for MPPI update
        self.update_mean_logits(costs.squeeze(-1))  # shape (num_samples, batch_size)
        return self._get_optimal_action(tensordict)

    def _get_optimal_action(self, tensordict):
        """
        Returns the optimal action for the first step of the current
        optimized mean trajectory for each batch element.

        Args:
            tensordict: The current observation/state. Not directly used for sampling
                        logic here but is standard for planners. It's cloned.
                        You'll use this when you evaluate the samples in your sim.

        Returns:
            torch.Tensor: Optimal action, shape (batch_size, action_dim).
        """
        # Get the optimal weights for the first step
        optimal_weights = self.get_optimal_action_weights()  # (batch_size, num_policy_options)
        # Get the actions from each policy
        actions = torch.stack([p(tensordict) for p in self.policies_callables], dim=0)  # (num_policy_options, batch_size, action_dim)
        # Compute the weighted sum of actions
        action = (actions * optimal_weights.T).sum(dim=0)
        return action

    def _get_rollout_weights(self):
        """
        Performs the sampling step of MPPI.
        Generates `num_samples` sequences of policy-blending weights.

        Args:
            tensordict: The current observation/state. Not directly used for sampling
                        logic here but is standard for planners. It's cloned.
                        You'll use this when you evaluate the samples in your sim.

        Returns:
            torch.Tensor: Sampled policy weight sequences, shape
                          (num_samples, rollout_steps, num_policy_options).
                          These are the weights you'll use to linearly combine your policies.
        """
        # 1. Shift mean logits for receding horizon control
        # The mean logits from the previous timestep are shifted, and the last step is re-initialized.
        shifted_logits = self.mean_logits[1:].clone()
        # Initialize the new last step logits to zeros (or another strategy)
        new_last_step_logits = torch.zeros(1, self.env.n_robots, self.num_policy_options, device=self.device)
        self.mean_logits = torch.cat((shifted_logits, new_last_step_logits), dim=0)
        logit_perturbations = (
            torch.randn(self.num_samples, self.rollout_steps, self.env.n_robots, self.num_policy_options, device=self.device) * self.noise_sigma
        )
        # Store perturbations for the update_mean_logits step
        self.last_logit_perturbations = logit_perturbations
        # 3. Create sampled logits
        # Broadcast mean_logits: (1, rollout_steps, 3) + (num_samples, rollout_steps, 3)
        sampled_logits = self.mean_logits.unsqueeze(0) + self.last_logit_perturbations
        # 4. Convert sampled logits to policy weights using softmax
        # This ensures weights for the 3 policies sum to 1 and are non-negative at each step.
        # sampled_policy_weights shape: (num_samples, rollout_steps, num_policy_options)
        sampled_policy_weights = F.softmax(sampled_logits / self.temperature, dim=-1)
        # These sampled_policy_weights are what you'll use in your environment
        # to simulate the `num_samples` different trajectories.
        return sampled_policy_weights

    def update_mean_logits(self, costs: torch.Tensor):
        """
        Updates mean logits using costs for each batch element independently.

        Args:
            costs (torch.Tensor): Costs for each sample and batch element,
                                shape (num_samples, batch_size).
        """
        if self.last_logit_perturbations is None:
            raise RuntimeError("`__call__` must be invoked first.")
        if costs.shape != (self.num_samples, self.env.n_robots):
            raise ValueError(f"Expected costs shape ({self.num_samples}, {self.env.n_robots}), got {costs.shape}")
        costs = costs.to(self.device)

        # Compute exponent terms per sample and batch element
        exponent_terms = -costs / self.lambda_  # (num_samples, batch_size)
        max_exponents = torch.max(exponent_terms, dim=0)[0]  # (batch_size,)
        stable_exponent_terms = exponent_terms - max_exponents.unsqueeze(0)  # (num_samples, batch_size)
        importance_weights = torch.exp(stable_exponent_terms)  # (num_samples, batch_size)

        # Normalize weights per batch element
        sum_of_weights = torch.sum(importance_weights, dim=0)  # (batch_size,)
        epsilon = 1e-9
        normalized_weights = importance_weights / (sum_of_weights.unsqueeze(0) + epsilon)  # (num_samples, batch_size)

        # Compute update delta
        weighted_perturbations = (
            normalized_weights.unsqueeze(1).unsqueeze(-1) * self.last_logit_perturbations
        )  # (num_samples, 1, batch_size, 1) * (num_samples, rollout_steps, batch_size, num_policy_options)
        update_delta = torch.sum(weighted_perturbations, dim=0)  # (rollout_steps, batch_size, num_policy_options)

        self.mean_logits += update_delta
        self.last_logit_perturbations = None

    def get_optimal_action_weights(self):
        """
        Returns the policy blending weights for the *first step* of the current
        optimized mean trajectory for each batch element.

        Returns:
            torch.Tensor: Optimal policy blending weights, shape
                          (batch_size, num_policy_options).
        """
        # Get the mean logits for the first step of the planned horizon
        first_step_mean_logits = self.mean_logits[0, :, :]  # (batch_size, num_policy_options)

        # Convert these logits to policy weights using softmax with temperature
        optimal_weights = F.softmax(first_step_mean_logits / self.temperature, dim=-1)

        return optimal_weights

In [9]:
steps = 10
num_samples = 1
lambda_ = 1.0
noise_sigma = 0.1
temperature = 1.0
planner = MultiPolicyMPPIPlanner(
    policy_obj=actor_value_wrapper,
    vecnorm_obj=vecnorm,
    weights=p_sds,
    vecnorm_weights=vecnorm_sds,
    env=env,
    device=device,
    rng=rng,
    rollout_steps=steps,
    num_samples=num_samples,
    lambda_=lambda_,
    noise_sigma=noise_sigma,
    temperature=temperature,
)

OrderedDict({'_extra_state': {'LocalStateVector_sum': tensor[15] x∈[-492.177, 1.336e+04] μ=3.494e+03 σ=5.413e+03 cuda:0, 'LocalStateVector_ssq': tensor[15] x∈[11.335, 8.346e+03] μ=2.141e+03 σ=3.179e+03 cuda:0, 'LocalStateVector_count': tensor[1] cuda:0 [2.560e+04], 'reward_sum': tensor[1] cuda:0 [5.083e+03], 'reward_ssq': tensor[1] cuda:0 [1.310e+04], 'reward_count': tensor[1] cuda:0 [2.560e+04]}})
OrderedDict({'_extra_state': {'LocalStateVector_sum': tensor[15] x∈[-481.560, 2.411e+04] μ=7.061e+03 σ=1.065e+04 cuda:0, 'LocalStateVector_ssq': tensor[15] x∈[9.653, 1.418e+04] μ=4.104e+03 σ=5.857e+03 cuda:0, 'LocalStateVector_count': tensor[1] cuda:0 [5.120e+04], 'reward_sum': tensor[1] cuda:0 [-4.908e+03], 'reward_ssq': tensor[1] cuda:0 [2.139e+04], 'reward_count': tensor[1] cuda:0 [5.120e+04]}})
OrderedDict({'_extra_state': {'LocalStateVector_sum': tensor[15] x∈[-643.012, 2.568e+04] μ=7.533e+03 σ=1.128e+04 cuda:0, 'LocalStateVector_ssq': tensor[15] x∈[8.526, 1.508e+04] μ=4.339e+03 σ=6.272

/home/korcadav/.venv/lib/python3.12/site-packages/torchrl/envs/transforms/transforms.py:6054: UserWarning: VecNorm wasn't initialized and the tensordict is not shared. In single process settings, this is ok, but if you need to share the statistics between workers this should require some attention. Make sure that the content of VecNorm is transmitted to the workers after calling load_state_dict and not before, as other workers may not have access to the loaded TensorDict.
  warnings.warn(


In [11]:
env.reset()
rollout = env.rollout(10000, planner)

2025-05-22 20:36:14,304 [environment][INFO]: Resetting environment (env.py:271)
2025-05-22 20:36:14,314 [environment][INFO]: Resetting environment (env.py:271)


KeyboardInterrupt: 

In [ ]:
log = log_from_eval_rollout(rollout)
log

{'eval/mean_step_reward': -0.0774034857749939,
 'eval/max_step_reward': 0.5402874946594238,
 'eval/min_step_reward': -7.52587366104126,
 'eval/mean_step_count': 145.0,
 'eval/max_step_count': 145.00009155273438,
 'eval/min_step_count': 144.99990844726562,
 'eval/pct_succeeded': 0.0,
 'eval/pct_failed': 0.0625,
 'eval/pct_truncated': 0.9375}

In [ ]:
terrain_cfg = env.terrain_cfg
trajectories = rollout["next", "curr_state"]["x"].cpu()
z_normalized = terrain_cfg.z_grid / terrain_cfg.z_grid.max()
z_normalized = z_normalized.cpu()
x_grid = terrain_cfg.x_grid.cpu()
y_grid = terrain_cfg.y_grid.cpu()
col = 4
row = 4
single_size = 5
indices_to_show = range(0, row * col)
fig, ax = plt.subplots(row, col, figsize=(col * single_size, row * single_size), dpi=200)
if ax.ndim == 1:
    ax = ax.reshape(1, -1)
j = 0
for i in indices_to_show:
    row_idx = j // col
    col_idx = j % col
    j += 1
    start_pos = env.start[i].x.cpu()
    goal_pos = env.goal[i].x.cpu()
    ax[row_idx, col_idx].grid(False)
    ax[row_idx, col_idx].axis("off")
    ax[row_idx, col_idx].contourf(
        x_grid[i],
        y_grid[i],
        z_normalized[i],
        cmap="gray",
        levels=10,
    )
    done_i = rollout["next", "done"][i].cpu()
    not_done = ~done_i
    traj_i = trajectories[i, not_done.squeeze(), :2]
    ax[row_idx, col_idx].plot(
        traj_i[:, 0],
        traj_i[:, 1],
        color="aquamarine",
        linewidth=4,
        label="Trajectory",
    )
    ax[row_idx, col_idx].scatter(
        start_pos[None, 0],
        start_pos[None, 1],
        color="red",
        marker="x",
        s=300,
        label="Start",
        lw=3,
    )
    ax[row_idx, col_idx].scatter(
        goal_pos[None, 0],
        goal_pos[None, 1],
        color="green",
        marker="x",
        s=300,
        label="Goal",
        lw=3,
    )
    fig.tight_layout()

ax[0, -1].legend(
    fontsize=14,
    loc="upper right",
    borderpad=0.5,  # padding around the legend
    labelspacing=0.8,  # vertical space between entries
    handletextpad=1.0,  # space between handle and text
)

RuntimeError: Failed to process string with tex because latex could not be found

Error in callback <function _draw_all_if_interactive at 0x7f1bdf203a60> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 4000x4000 with 16 Axes>